In [9]:
from langchain_ollama import ChatOllama,OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.messages import HumanMessage, BaseMessage,AIMessage
from langchain_core.tools import tool
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.types import interrupt, Command

from langgraph.graph import StateGraph, START,END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from typing import Annotated, TypedDict,List
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [4]:
llm = ChatOllama(model='qwen2:7b')

In [5]:
class ChatState(TypedDict):

    messages: Annotated[list[BaseMessage], add_messages]

In [8]:
def chat_node(state: ChatState):

    decision = interrupt({
        "type": "approval",
        "reason": "Model is about to answer a user question.",
        "question": state["messages"][-1].content,
        "instruction": "Approve this question? yes/no"
    })
    
    if decision["approved"] == 'no':
        return {"messages": [AIMessage(content="Not approved.")]}

    else:
        response = llm.invoke(state["messages"])
        return {"messages": [response]}

In [10]:
graph = StateGraph(ChatState)

graph.add_node("chat", chat_node)

graph.add_edge(START, "chat")
graph.add_edge("chat", END)

checkpointer = MemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [11]:
config = {"configurable": {"thread_id": '1'}}

initial_input = {
        "messages":[("user", "Explain gradient descent in very simple terms.")]
}

result = workflow.invoke(initial_input, config=config)

In [13]:
message = result['__interrupt__'][0].value
message

{'type': 'approval',
 'reason': 'Model is about to answer a user question.',
 'question': 'Explain gradient descent in very simple terms.',
 'instruction': 'Approve this question? yes/no'}

In [20]:
user_input = input(f"\nBackend message - {message} \n Approve this question? (y/n): ")

In [21]:
final_result = workflow.invoke(
    Command(resume={"approved": user_input}),
    config=config,
)

In [22]:
print(final_result["messages"][-1].content)

Gradient Descent is an optimization algorithm used to find the best parameters (such as coefficients or weights) for a model so that it minimizes a specific cost function. In simpler terms, it helps us find the lowest point in a valley by looking at the slope of the terrain around us.

Imagine you're standing on a hilly landscape and trying to find the lowest point. Gradient descent works like this:

1. **Choose an initial starting point**: This represents your first guess for where the lowest point might be.
2. **Look around**: At your current position, check the slope of the hill (this is called the gradient). The direction with the most negative slope indicates downhill and thus toward the lowest point.
3. **Move downhill**: Take a small step in that downhill direction. This new position becomes your next guess for where the lowest point might be.
4. **Repeat steps 2 & 3**: Continue checking the slope around you (the gradient) and moving in the downhill direction until you can't fin